In [11]:
# !python -m pip install geopandas
import pandas as pd
import numpy as np
import geopandas as gpd
from scipy.stats import zscore


In [12]:
#reading rheast and extracting districts
tamsat= pd.read_csv('data/rheas/zmb_n_25_chirps__newmask_1999_2023_maize.csv')
preharvest = pd.read_csv(r'data/preharvest/Zambia_District _Maize_Forecasting_2011_2022.xlsx - 2022.csv', header=1)
postharvest=pd.read_csv('data/postharvest/postharvest_cleaned.csv')


In [13]:
# remove spaces
preharvest.columns=preharvest.columns.str.lower()
preharvest.columns=preharvest.columns.str.strip()
preharvest.columns=preharvest.columns.str.replace(' ', '', regex=True)#remove space in between

# remove all white spaces, turn to lower case and
preharvest.district=preharvest.district.str.lower()
preharvest.district = preharvest.district.str.strip()
preharvest.district=preharvest.district.str.replace('-', '', regex=True)#remove hyphen
# preharvest.district=preharvest.district.str.replace(' ', '', regex=True)#removes spaces anywhere

# remove spaces in other columns as well
preharvest.planted_ha = preharvest.planted_ha.str.strip()
preharvest.planted_ha=preharvest.planted_ha.str.replace(',', '', regex=True)#removes spaces anywhere

preharvest.harvested_ha = preharvest.harvested_ha.str.strip()
preharvest.harvested_ha=preharvest.harvested_ha.str.replace(',', '', regex=True)#removes spaces anywhere

preharvest.production_mt = preharvest.production_mt.str.strip()
preharvest.production_mt=preharvest.production_mt.str.replace(',', '', regex=True)#removes spaces anywhere

In [14]:
# drop province
preharvest=preharvest.drop('province', axis=1)

In [15]:
# clean preharvest survey data
print(len(preharvest))
#drop duplicate inputs
preharvest = preharvest.drop_duplicates()
print(len(preharvest))
preharvest

996
996


,district,planted_ha,harvested_ha,production_mt,yield(mt/ha),year
0,chibombo,69254,52256,148888,2.15,2011
1,kabwe,5526,5084,25637,4.64,2011
2,kapirimposhi,48651,37962,103335,2.12,2011
3,mkushi,34536,28538,130000,3.76,2011
4,mumbwa,35944,29544,95568,2.66,2011
...,...,...,...,...,...,...
991,mongu,12433,9711,11025,0.89,2022
992,senanga,19309,10295,11132,0.58,2022
993,sesheke,17756,10282,18585,1.05,2022
994,shangombo,20788,15981,24396,1.17,2022


In [16]:
# convert appropriate columns to numeric
preharvest.planted_ha=pd.to_numeric(preharvest.planted_ha)
preharvest.harvested_ha=pd.to_numeric(preharvest.harvested_ha)
preharvest.production_mt=pd.to_numeric(preharvest.production_mt)
preharvest['yield(mt/ha)']=pd.to_numeric(preharvest['yield(mt/ha)'])

In [17]:

#calculate yield
preharvest = preharvest.groupby(['district','year'],as_index=False).sum()
preharvest['yield'] = preharvest['production_mt'] / preharvest['planted_ha']
preharvest



,district,year,planted_ha,harvested_ha,production_mt,yield(mt/ha),yield
0,chadiza,2011,21879,21293,42951,1.96,1.963115
1,chadiza,2012,23894,21453,51340,2.15,2.148657
2,chadiza,2013,24901,23476,50022,2.01,2.008835
3,chadiza,2014,25395,22957,61135,2.41,2.407364
4,chadiza,2015,25379,21825,51345,2.02,2.023129
...,...,...,...,...,...,...,...
893,zambezi,2018,4682,3941,7595,1.62,1.622170
894,zambezi,2019,5885,3990,6973,1.18,1.184877
895,zambezi,2020,7193,6031,12153,1.69,1.689559
896,zambezi,2021,6350,5533,18935,2.98,2.981890


In [18]:
# #calculate z scores of yield and flag any anomalies
df1 = []
# # preharvest = list(preharvest.reset_index())
for d in list(preharvest.district.unique()):
    df = preharvest.loc[preharvest['district']==d].copy()
    df['zyield'] = zscore(df['yield'])
    df1.append(df)
    df2=pd.concat(df1)


In [19]:
# add a flag to the dataframe
df2['flag'] = 0
df2['flag'].loc[(((df2['zyield']>1.96).copy())|((df2['zyield']<-1.96).copy()))]= 1
df2['flag'].loc[(((df2['zyield']>2.58).copy())|((df2['zyield']<-2.58).copy()))] = 2
df2

C:\Users\kasit\AppData\Local\Temp\ipykernel_13156\413039638.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['flag'].loc[(((df2['zyield']>1.96).copy())|((df2['zyield']<-1.96).copy()))]= 1
C:\Users\kasit\AppData\Local\Temp\ipykernel_13156\413039638.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['flag'].loc[(((df2['zyield']>2.58).copy())|((df2['zyield']<-2.58).copy()))] = 2


,district,year,planted_ha,harvested_ha,production_mt,yield(mt/ha),yield,zyield,flag
0,chadiza,2011,21879,21293,42951,1.96,1.963115,-0.513418,0
1,chadiza,2012,23894,21453,51340,2.15,2.148657,-0.050325,0
2,chadiza,2013,24901,23476,50022,2.01,2.008835,-0.399306,0
3,chadiza,2014,25395,22957,61135,2.41,2.407364,0.595381,0
4,chadiza,2015,25379,21825,51345,2.02,2.023129,-0.363629,0
...,...,...,...,...,...,...,...,...,...
893,zambezi,2018,4682,3941,7595,1.62,1.622170,-0.559302,0
894,zambezi,2019,5885,3990,6973,1.18,1.184877,-1.606755,0
895,zambezi,2020,7193,6031,12153,1.69,1.689559,-0.397884,0
896,zambezi,2021,6350,5533,18935,2.98,2.981890,2.697649,2


In [20]:
# save to csv
# df2.to_csv('/data/preharvest/zambia_preharvest_data_clean.csv')